In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일 자동으로 루트에서 찾아서 로드
api_key = os.getenv("OPENAI_API_KEY")


In [4]:

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings  # 또는 HuggingFaceEmbeddings

# PDF 로딩
loader = PyPDFLoader("../data/sleep_performance.pdf")
documents = loader.load_and_split()

# 임베딩 생성
embedding = OpenAIEmbeddings()  # .env에 API 키 필요!
vectorstore = FAISS.from_documents(documents, embedding)

# 저장
vectorstore.save_local("vectorstores/sleep_rag")

from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# 벡터 DB 로딩
vectorstore = FAISS.load_local(
    "vectorstores/sleep_rag",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)


In [ ]:
retriever = vectorstore.as_retriever()

# LLM 설정
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
def generate_meal_plan(request_data: dict) -> str:
    # 필수 데이터 추출 및 기본값 설정
    user_id = request_data.get("user_id", "알 수 없음")
    goal = request_data.get("goal", "다이어트")
    diseases = request_data.get("diseases", [])
    records = request_data.get("records", [])
    sleep = "정보 없음"

    if records:
        sleep = records.get("sleep", "정보 없음")

    diseases_str = ", ".join(diseases) if diseases else "없음"

    sleep_status_query = f"수면 시간 {sleep}시간에 따른 식단 조언 및 건강 영향"
    if isinstance(sleep, (int, float)):
        if sleep < 6:
            sleep_status_query = f"수면 부족({sleep}시간) 시 회복에 도움 되는 식단"
        elif 6 <= sleep <= 8.5: # 7~9시간 범위는 적정 수면으로 간주, 유연하게 조절
            sleep_status_query = f"적정 수면({sleep}시간)을 위한 최적의 식단 가이드"
        else: 
            sleep_status_query = f"과도한 수면({sleep}시간) 시 식단 고려사항"
    
    print(f"\n수면 RAG 검색 쿼리: '{sleep_status_query}'")
    retrieved_sleep_docs = retriever.invoke(sleep_status_query)
    
    # 검색된 문서 내용을 프롬프트에 포함할 문자열로 변환
    sleep_context = "\n".join([doc.page_content for doc in retrieved_sleep_docs])
    if not sleep_context.strip(): # 검색 결과가 없을 경우 대비
        sleep_context = "사용자의 수면 시간에 따른 특정 식단 가이드라인이 검색되지 않았습니다. 일반적인 건강 식단을 제안합니다."

    print(f"--- 검색된 수면 관련 문서 ({len(retrieved_sleep_docs)}개) ---")
    for i, doc in enumerate(retrieved_sleep_docs):
        print(f"Doc {i+1}: {doc.page_content[:100]}...") # 내용의 앞부분만 출력

    prompt_template = ChatPromptTemplate.from_messages(
        [
            ("system", """당신은 개인 맞춤형 영양사 AI입니다. 다음 사용자 정보와 건강 목표, 질병 이력, 그리고 제공된 수면시간을 바탕으로 아침, 점심, 저녁 식단을 구체적으로 제안해주세요. 
                        식단은 건강하며 균형 잡히고, 특히 사용자의 목표 달성과 질병 관리에 도움이 되며, 수면 상태를 고려한 내용이어야 합니다. 응답은 반드시 [응답 형식]에 맞춰주세요."""),
            ("user", """
            ---
            [사용자 정보]
            사용자 ID: {user_id}
            건강 목표: {goal}
            현재 앓고 있는 질병: {diseases}
            수면 시간: {sleep}시간

            [수면 관련 가이드라인 (RAG 검색 결과)]
            {sleep_context}
            ---

            [요청]
            위 정보를 종합하여 아침, 점심, 저녁 식단 예시를 구체적으로 제안해주세요. 각 식단에는 메뉴와 간단한 설명을 포함해주세요.
            식단은 사용자의 {goal}라는 목표와 {diseases} 질병 관리에 도움이 되어야 합니다.

            [응답 형식]
            식단:
            - 아침: [아침 식단 메뉴와 양(gram 수 혹은 개수) 및 설명]
            - 점심: [점심 식단 메뉴와 양(gram 수 혹은 개수) 및 설명]
            - 저녁: [저녁 식단 메뉴와 양(gram 수 혹은 개수) 및 설명]
            """)
        ]
    )

    prompt_data = {
        "user_id": user_id,
        "goal": goal,
        "diseases": diseases_str,
        "sleep": sleep,
        "sleep_context": sleep_context
    }

    chain = prompt_template | llm | StrOutputParser()
    print("\n식단 생성 AI 호출 중...")
    response = chain.invoke(prompt_data)

    return response

In [10]:
# --- 사용 예시 (API 엔드포인트에서 이 함수를 호출) ---
if __name__ == "__main__":
    # 이 부분은 Spring Boot 백엔드로부터 받아올 실제 request body를 모방합니다.
    sample_request_body = {
        "user_id": 1,
        "goal": "근력 향상",
        "diseases": ["당뇨", "고혈압"],
        "records": {
                "date": "2025-06-27",
                "sleep": 6.5, # 수면 시간
                "weight": 70.2,
                "fat": 17.5,
                "muscle": 32.1,
                "bmr": 1580,
                "bmi": 23.5,
                "vai": 1.2
            },
        "todolists": [
            {
                "date": "2025-06-20",
                "items": [
                    {"todo": "유산소 50분", "complete": True},
                    {"todo": "스트레칭 10분", "complete": False}
                ]   
            },
            {
                "date": "2025-06-21",
                "items": [
                    {"todo": "윗몸 일으키기 30개", "complete": True},
                    {"todo": "스트레칭 10분", "complete": True}
                ]   
            },
            {
                "date": "2025-06-22",
                "items": [
                    {"todo": "런지 30개", "complete": True},
                    {"todo": "스트레칭 10분", "complete": True}
                ]   
            },
            {
                "date": "2025-06-23",
                "items": [
                    {"todo": "스쿼트 40개", "complete": True},
                    {"todo": "스트레칭 10분", "complete": False}
                ]   
            },
            {
                "date": "2025-06-24",
                "items": [
                    {"todo": "스쿼트 30개", "complete": True},
                    {"todo": "스트레칭 10분", "complete": True}
                ]   
            },
            {
                "date": "2025-06-25",
                "items": [
                    {"todo": "스쿼트 30개", "complete": False},
                    {"todo": "스트레칭 10분", "complete": True}
                ]   
            },
            {
                "date": "2025-06-26",
                "items": [
                    {"todo": "스쿼트 30개", "complete": False},
                    {"todo": "스트레칭 10분", "complete": False}
                ]   
            }
        ],
        "prompt": "오늘은 등 운동을 할거야",
        "place": "헬스장"
    }

    try:
        print("수면 RAG 고려 식단 생성 기능 테스트 시작...")
        meal_plan_result = generate_meal_plan(sample_request_body)
        
        print("\n--- 생성된 최종 식단 ---")
        print(meal_plan_result)

    except FileNotFoundError as e:
        print(f"\nError: {e}")
        print("Please ensure you have run 'rag_retriever_builder.py' to create the 'vectorstores/sleep_rag' index.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        print("Please check your OpenAI API key and network connection.")

수면 RAG 고려 식단 생성 기능 테스트 시작...

수면 RAG 검색 쿼리: '적정 수면(6.5시간)을 위한 최적의 식단 가이드'
--- 검색된 수면 관련 문서 (4개) ---
Doc 1: Kong et al. 10.3389/fphys.2025.1544286
TABLE 1 Basic characteristics of the included studies.
Popula...
Doc 2: Kong et al. 10.3389/fphys.2025.1544286
TABLE 1 ( Continued) Basic characteristics of the included st...
Doc 3: Kong et al. 10.3389/fphys.2025.1544286
TABLE 1 ( Continued) Basic characteristics of the included st...
Doc 4: vs healthy non-athletes), sleep deprivation type (TSD vs PSDB vs
PSDE vs P), and testing time (a.m. ...

식단 생성 AI 호출 중...

--- 생성된 최종 식단 ---
식단:
- 아침: 
  - 스크램블 에그와 아보카도 토스트
    - 스크램블 에그: 2개
    - 아보카도 토스트: 1조각
    - 설명: 고단백, 고지방의 스크램블 에그와 아보카도는 근력 향상에 도움을 주는 영양소를 풍부하게 함유하고 있습니다. 토스트는 탄수화물을 공급하여 에너지를 제공합니다.

- 점심: 
  - 그릴 치킨 샐러드
    - 그릴 치킨: 100g
    - 채소 (양상추, 토마토, 오이): 적당량
    - 올리브 오일 드레싱: 1큰술
    - 설명: 치킨은 고품질 단백질을 제공하며, 채소는 비타민과 미네랄을 풍부하게 함유하고 있어 질병 관리에 도움을 줍니다.

- 저녁: 
  - 오븐 구운 연어와 채소 볶음
    - 연어: 1조각 (150g)
    - 채소 (브로콜리, 당근, 양파): 적당량
    - 올리브